## Assignment 2

*100 points (8% of course grade)*</br>
*Assigned: Sun, May 26th*</br>
**Due: Fri, Jun 7th, 23:59**

This homework should be done in parts as soon as (< 0.5 week) relevant topics are covered in lectures. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically, including this Notebook (A2.ipynb). Please follow the submission instructions for each problem carefully.

In this assignment, you need to solve two problems. In Problem 1, you will write a few SQL queries to query the beers database. In Problem 2, you will answer two questions in database design theory.

## Setup your PostgreSQL

You will need this setup to create a database on your machine and to test your queries. Please follow our [setup instructions](https://canvas.sfu.ca/courses/84701/pages/postgresql-setup-instruction-for-assignment-2) on Canvas.

### Problem 1: Query with SQL (64%)

Consider again the beer drinker's database from Assignment 1 with following schema (key columns underlined).

- drinker (<u>name</u>, address)
- bar (<u>name</u>, address)
- beer (<u>name</u>, brewer)
- frequents (<u>drinker</u>, <u>bar</u>, times_a_week)
- likes (<u>drinker</u>, <u>beer</u>)
- serves (<u>bar</u>, <u>beer</u>, price)


#### **Preliminary**

After you finish the PostgreSQL setup, you will be able to run the PostgreSQL's interpreter in your command line tools by running `psql beers -U [your username]` in your terminal or command line. You may also use pgAdmin instead of the command line tool.

Same as assignment 1, you will need to type your SQL queries in the cells below.


Now your homework question is to write SQL queries to answer following questions. One major difference between SQL and relational algebra queries is the *bag semantics*: you may need to use DISTINCT in your SELECT statement or aggregate functions to deduplicate results or intermediate results.

Please fill your answer in each cell (and **ONLY the query**) and **DO NOT add or remove** any cells to make the TAs' life easier in evaluating your queries. Questions (1)-(4) are worth 6 points each; (5)-(9) are worth 8 points each.


#### 0. (example) Find names of all bars that Eve frequents.

In [ ]:
/* input your answer in this cell: */

SELECT bar FROM frequents WHERE drinker = 'Eve';

#### 1. Find names of bars that serve either Amstel or Corona at price higher than \\$2.

In [ ]:
/* input your answer in this cell: */
SELECT DISTINCT Bar 
	from Serves 
	where ((Beer='Corona' or Beer='Amstel') 
	and Price > 2);

#### 2. Find the names of all drinkers that do not like Corona but frequent a bar that serves Corona.

In [ ]:
/* input your answer in this cell: */
SELECT DISTINCT frequents.drinker
FROM serves
JOIN frequents ON serves.bar = frequents.bar
WHERE serves.beer = 'Corona'
  AND frequents.drinker NOT IN (
    SELECT drinker
    FROM Likes
    WHERE beer = 'Corona'
);

#### 3. Find the names of all bars that serve at least 5 beers


In [ ]:
/* input your answer in this cell: */
SELECT Bar
FROM Serves
GROUP BY Bar
HAVING COUNT(DISTINCT Beer) >= 5;

#### 4. Find the pair of drinkers who frequent bars the same total number of times per week. Don't list (drinkerA, drinkerA). Only list pairs (drinkerA, drinkerB) where drinkerA < drinkerB  in the answer; don't list (drinkerB, drinkerA) again.

In [ ]:
/* input your answer in this cell: */
SELECT First_drinker, Second_drinker
FROM (
    SELECT Drinker AS First_drinker, SUM(times_a_week) AS Total_visits_a
    FROM Frequents
    GROUP BY Drinker
) AS a
JOIN (
    SELECT Drinker AS Second_drinker, SUM(times_a_week) AS Total_visits_b
    FROM Frequents
    GROUP BY Drinker
) AS b ON Total_visits_a = Total_visits_b
WHERE (First_drinker < Second_drinker);

#### 5. Find names of all drinkers who frequent *only* those bars that serve *only* beers they like.

In [ ]:
/* input your answer in this cell: */
SELECT name
FROM drinker
EXCEPT
SELECT L.drinker
FROM (frequents NATURAL JOIN serves) AS people_and_beers
JOIN likes L ON L.drinker = people_and_beers.drinker
WHERE people_and_beers.beer NOT IN (
    SELECT beer
    FROM likes
    WHERE drinker = people_and_beers.drinker
);


#### 6. Find all (bar1, bar2) pairs where the set of beers served at bar1 is a proper subset of the those served at bar2; i.e., bar2 serves every beer that bar1 serves and plus some more.

In [ ]:
/* input your answer in this cell: */
SELECT DISTINCT A.Bar AS Bar1, B.Bar AS Bar2
FROM Serves A
JOIN Serves B ON A.Beer = B.Beer
WHERE A.Bar <> B.Bar
AND NOT EXISTS (
    SELECT 1
    FROM Serves C
    WHERE C.Bar = A.Bar
    AND C.Beer NOT IN (
        SELECT Beer
        FROM Serves D
        WHERE D.Bar = B.Bar
    )
)
AND EXISTS (
    SELECT 1
    FROM Serves E
    WHERE E.Bar = B.Bar
    AND E.Beer NOT IN (
        SELECT Beer
        FROM Serves F
        WHERE F.Bar = A.Bar
    )
);

#### 7. Calculate some statistics for each bar: (1) total number of drinkers who frequent it, (2) average price of beers it serves, and (3) name of the drinker who frequents it the maximum number of times a week (the most regular customer!). i.e., your query should output total_num_drinker, avg_price, most_reg_drinker
In case of ties in (3), rank the drinkers by the alphabetical order or their names. Sort the output by the number of drinkers (in descending order), in case of ties,
- sort by bar in alphabetical order. You need to list every bar, even if it
is not frequented by anyone
- (show 0 as the total number of drinkers in this case and NULL as the most
regular customer) or
- serves no beers (show NULL as average price in this case).

In [ ]:
/* input your answer in this cell: */
SELECT
    name,
    COALESCE((SELECT COUNT(DISTINCT drinker) FROM frequents WHERE bar = bar.name), 0) AS total_num_drinkers,
    COALESCE((SELECT AVG(price) FROM serves WHERE bar = bar.name), NULL) AS avg_price,
    COALESCE((SELECT drinker FROM frequents WHERE bar = bar.name ORDER BY times_a_week DESC, drinker LIMIT 1), NULL) AS most_reg_drinker
FROM
    bar
ORDER BY
    total_num_drinkers DESC,
    name ASC;

#### 8. Find, for each beer served in at least 2 bars that frequented by drinkers whose name starts with 'B', the difference of maximum and minimum (price gap) and *average price* of the beers in all serving bars, order by beer name. I.e., your query should output beer, price_gap, and avg_price

In [ ]:
/* input your answer in this cell: */
SELECT
    beer,
    ((SELECT MAX(price) FROM serves WHERE serves.beer = main.beer) - (SELECT MIN(price) FROM serves WHERE serves.beer = main.beer)) AS price_gap,
    (SELECT AVG(price) FROM serves WHERE serves.beer = main.beer) AS avg_price
FROM (
    SELECT beer, bar
    FROM serves
    WHERE bar IN (
        SELECT bar
        FROM frequents
        WHERE drinker LIKE 'B%'
    )
) main
GROUP BY beer
HAVING COUNT(DISTINCT bar) >= 2
ORDER BY beer;

#### 9. Find, for each beer, its lowest serving price and the bar(s) serving it at this price. The output should be a list of (beer, price, bar) triples. If some beer B is not served anywhere, you should still output (B, NULL, NULL).


In [ ]:
/* input your answer in this cell: */
WITH MinPrices AS (
    SELECT beer, MIN(price) AS min_price
    FROM serves
    GROUP BY beer
),
BeerAndMinPrices AS (
    SELECT MP.beer, MP.min_price, S.bar
    FROM MinPrices MP
    JOIN serves S ON MP.beer = S.beer AND MP.min_price = S.price
)
SELECT B.name AS beer, BMP.min_price AS price, BMP.bar AS bar
FROM beer B
LEFT JOIN BeerAndMinPrices BMP ON B.name = BMP.beer
ORDER BY B.name;

## Problem 2: Database design theory (36% = 16% + 20%)

#### 1. Consider a relation R with five attributes ABCDE. You are given the following dependencies: A -> D; DE -> C; CB -> A. 
1. For each FD $X \rightarrow Y$, compute the closure $X^+$
2. List all keys in R.
3. Is R in 3NF? Explain.
4. Is R in BCNF? Explain.

`Write your answers for 1,2,3,4 in this cell`

1. $A^+$ = { A } applying A -> D, $A^+$ = {A, D}
There is no further attributes to be added so it is $A^+$ = {A, D}

$DE^+$  = {D,E} applying DE -> C, $DE^+$ = {D,E,C}
There is no further attributes to be added so it is $DE^+$ = {D,E,C}

$CB^+$ = {C,B} applying CB -> A, $CB^+$ = {C,B,A}
A -> D, $CB^+$ = {C,B,A,D} and using DE -> C and having the D and E in $CB^+$ already,
$CB^+$ = {C,B,A,D,E}

2. keys for the R are {C,B,E} {D,E,B} {A,E,B}

3. relation is in 3NF if every non-trivial functional dependency X -> Y
X is a superkey or Y is a prime attribute. For our FDs 
A->D A is not the superkey and D is not a prime attribute. (violate 3NF)
DE->C DE is not a superkey and C is not prime attribute. (violate 3NF)
CB->A CB is a superkey. (does notviolate 3NF)
Since not every FD meets the 3NF requirement, R is not in 3NF

4. R is not in BCNF because for relation that is in BCNF, for every non trival functional dependency X->Y X is superkey.

for our given FD,
A->D A is not superkey
DE->C DE is not the superkey
CB->A CB is superkey

although it meets one CB->A but since all FD does not meet BCNF requirement, R is not in BCNF

#### 2. Consider the following simplified table storing information about Pals, jobs, and production structures in Palworld: R(pid, sid, pal, structure_name, work_speed, job_type, food_consumption) and a set of functional dependencies:
* FD1: pid, sid -> work_speed
* FD2: sid -> structure_name, job_type
* FD3: pid -> pal, food_consumption
* FD4: structure_name -> job_type

`Decompose the schema into BCNF by (1) filling out the steps below, and (2) writing the names of the
relations in the final solution. Note: You may not need all four steps in your decompositions.`

**Step 1**
- Initial table: `R(pid, sid, pal, structure_name, work_speed, job_type, food_consumption)`
- Violating FD $X \rightarrow Y$: sid -> structure_name, job_type
- Schema of new table-1: R1(sid, structure_name, job_type)
the table is from the functional dependncy sid -> structure_name, job_type
- Schema of new table-2: R2(pid,sid,pal,work_speed,food_consumption)
the table is from removing the structure_name and the job_type from the original table since they are now represented in the R1

**Step 2**
- Initial table: R2 (pid, sid, pal, work_speed, food_consumption)
- Violating FD $X \rightarrow Y$: FD1 : pid,sid->work_speed
- Schema of new table-1: R3(pid,sid,work_speed)
the table is from the functional dependency pid, sid ->work_speed
- Schema of new table-2: R4(pid,sid,pal,food_consumption)
the table is from removing work_speed from the R2

**Step 3**
- Initial table: R4 (pid,sid,pal,food_consumption)
- Violating FD $X \rightarrow Y$: FD3 pid->pal, food_consumption
- Schema of new table-1: R5 (pid,pal,food_consumption)
the table is from the functional dependency pid->pal,food_consumption
- Schema of new table-2: R6 (pid, sid)
the table is made from removing pal and food_consumption from R4

**Step 4**
- Initial table: R1(sid,structure_name,job_type)
- Violating FD $X \rightarrow Y$: FD4 structure_name -> job_type
- Schema of new table-1: R7 (structure_name, job_type) 
the table is made from the functional dependency structure_name -> job_type
- Schema of new table-2: R8 (sid, structure_name) 
the table is from by removing job_type from R1

**Names of relations in the final solution:**
R7 (structure_name,job_type) the structure_name is a superkey
R8 (sid,structure_name) sid is a superkey
R3 (pid,sid,work_speed) pid,sid is superkey
R5 (pid,pal,food_consumption) pid is superkey
R6 (pid,sid) pid,sid is superkey

it meets all the requirement for the BCNF requirement therefore it is BCNF

## Submission instruction

1. For problem 1, answer the questions (1)-(9) in the Markdown cells

2. For problem 2.1, answer questions 1,2,3,4 in the given cell.

2. For problem 2.2, fill the steps you need after the `:` and write down the names of relations in the final solution

3. Compress your A2.ipynb (this file) into A2.zip and submit on Canvas